# External Trade Generation from Signals

This notebook demonstrates how to automatically generate external trades from various signal types.

## Overview

The framework provides utilities to convert signals into external trades while:
- Accounting for current portfolio positions
- Applying ADV constraints
- Simulating realistic execution (slippage, multiple fills)
- Supporting multiple signal types (weights, positions, deltas, scores)

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
from backtesting import (
    Backtester, BacktestConfig, DataManager,
    ExternalTradeGenerator, TradeGeneratorConfig,
    generate_external_trades_from_signals
)

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_rows', 20)

## Setup Data

In [ ]:
# Initialize data manager
data_manager = DataManager('../data')

# Get price data
prices_df = data_manager.get_prices()
print(f"Loaded prices for {len(prices_df.columns)} tickers from {prices_df.index[0]} to {prices_df.index[-1]}")
print(f"\nFirst few rows:")
print(prices_df.head())

## Example 1: Generate from Target Weights

Most common use case - specify desired portfolio weights.

In [ ]:
# Define target weights (as fraction of portfolio)
target_weights = {
    'AAPL': 0.30,   # 30% in Apple
    'MSFT': 0.25,   # 25% in Microsoft
    'GOOGL': 0.15,  # 15% in Google
    'TSLA': -0.10   # -10% (short Tesla)
}

# Current holdings
current_positions = {
    'AAPL': 100,   # Currently hold 100 shares of AAPL
    'MSFT': 200,   # 200 shares of MSFT
    'AMZN': 50     # 50 shares of AMZN (not in target - will be sold)
}

# Get prices for a specific date
date = prices_df.index[50]
close_prices = prices_df.loc[date].to_dict()
portfolio_value = 100000  # $100k portfolio

print(f"\nGenerating trades for {date.strftime('%Y-%m-%d')}")
print(f"Portfolio value: ${portfolio_value:,.0f}")
print(f"\nTarget weights: {target_weights}")
print(f"Current positions: {current_positions}")

# Generate trades
trades = generate_external_trades_from_signals(
    signals=target_weights,
    current_positions=current_positions,
    close_prices=close_prices,
    portfolio_value=portfolio_value,
    signal_type='weights',
    price_impact_bps=5.0,
    num_fills=1
)

print(f"\nGenerated trades:")
for ticker, trade_list in trades.items():
    total_qty = sum(t['qty'] for t in trade_list)
    avg_price = sum(t['qty'] * t['price'] for t in trade_list) / total_qty if total_qty != 0 else 0
    notional = total_qty * avg_price
    print(f"{ticker:6s}: {total_qty:6.0f} shares @ ${avg_price:7.2f} = ${notional:10,.0f}")

## Example 2: Generate from Target Positions

Specify exact share counts you want to hold.

In [ ]:
# Define target positions (in shares)
target_positions = {
    'AAPL': 500,   # Want to hold 500 shares
    'MSFT': 300,   # 300 shares
    'GOOGL': 100   # 100 shares
}

current_positions = {
    'AAPL': 300,   # Currently have 300
    'MSFT': 400,   # 400
    'GOOGL': 50    # 50
}

print(f"Target positions: {target_positions}")
print(f"Current positions: {current_positions}")

# Generate trades
trades = generate_external_trades_from_signals(
    signals=target_positions,
    current_positions=current_positions,
    close_prices=close_prices,
    portfolio_value=portfolio_value,
    signal_type='positions'
)

print(f"\nRequired trades:")
for ticker, trade_list in trades.items():
    total_qty = sum(t['qty'] for t in trade_list)
    action = "BUY" if total_qty > 0 else "SELL"
    print(f"{ticker:6s}: {action} {abs(total_qty):6.0f} shares")

## Example 3: Generate from Signal Scores

Convert alpha signals or z-scores to trades.

In [ ]:
# Alpha signals (higher = more bullish)
alpha_scores = {
    'AAPL': 2.5,    # Strong buy
    'MSFT': 1.2,    # Moderate buy
    'GOOGL': 0.3,   # Weak buy
    'TSLA': -1.8,   # Sell signal
    'AMZN': -0.5    # Weak sell
}

current_positions = {'AAPL': 100, 'TSLA': 50}

print(f"Alpha scores: {alpha_scores}")
print(f"Current positions: {current_positions}")

# Generate trades (weights proportional to |score|, direction from sign)
trades = generate_external_trades_from_signals(
    signals=alpha_scores,
    current_positions=current_positions,
    close_prices=close_prices,
    portfolio_value=portfolio_value,
    signal_type='scores',
    target_notional=100000  # Allocate $100k based on scores
)

print(f"\nGenerated trades from scores:")
for ticker, trade_list in trades.items():
    total_qty = sum(t['qty'] for t in trade_list)
    avg_price = sum(t['qty'] * t['price'] for t in trade_list) / total_qty if total_qty != 0 else 0
    notional = abs(total_qty * avg_price)
    weight = notional / portfolio_value
    print(f"{ticker:6s}: {total_qty:6.0f} shares (weight: {weight:5.1%})")

## Example 4: Multiple Fills (VWAP Simulation)

Generate multiple fills per ticker to simulate VWAP/TWAP execution.

In [ ]:
# Generate trades with 5 fills per ticker
trades = generate_external_trades_from_signals(
    signals={'AAPL': 0.30, 'MSFT': 0.20},
    current_positions={},
    close_prices=close_prices,
    portfolio_value=portfolio_value,
    signal_type='weights',
    price_impact_bps=5.0,
    num_fills=5  # Split into 5 fills
)

print("Generated trades with multiple fills (VWAP simulation):\n")
for ticker, trade_list in trades.items():
    print(f"{ticker}:")
    total_qty = 0
    total_cost = 0
    for i, trade in enumerate(trade_list, 1):
        qty = trade['qty']
        price = trade['price']
        cost = qty * price
        total_qty += qty
        total_cost += cost
        print(f"  Fill {i}: {qty:6.0f} shares @ ${price:7.2f}")
    
    vwap = total_cost / total_qty if total_qty != 0 else 0
    print(f"  Total:  {total_qty:6.0f} shares @ ${vwap:7.2f} (VWAP)\n")

## Example 5: Advanced Configuration

Use ExternalTradeGenerator class for fine-grained control.

In [ ]:
# Create custom configuration
config = TradeGeneratorConfig(
    price_impact_bps=10.0,        # Higher slippage (10 bps)
    use_random_fills=False,       # Deterministic prices
    num_fills_per_ticker=3,       # 3 fills per ticker
    min_trade_size=10.0,          # Minimum 10 shares
    round_lots=True,              # Round to whole shares
    max_adv_participation=0.05    # Max 5% of ADV
)

generator = ExternalTradeGenerator(config)

# Get ADV data (if available)
try:
    adv = data_manager.get_adv().loc[date].to_dict()
except:
    adv = None
    print("ADV data not available - using no ADV constraints")

# Generate trades
target_positions = {'AAPL': 1000, 'MSFT': 500, 'GOOGL': 300}
current_positions = {'AAPL': 500}

trades = generator.from_target_positions(
    target_positions=target_positions,
    current_positions=current_positions,
    close_prices=close_prices,
    adv=adv
)

print("\nTrades with custom config:")
for ticker, trade_list in trades.items():
    print(f"\n{ticker}:")
    for i, trade in enumerate(trade_list, 1):
        print(f"  Fill {i}: {trade['qty']:6.0f} shares @ ${trade['price']:7.2f}")

## Example 6: Multi-Day Trade Generation

Generate trades for multiple days from a signal time series.

In [ ]:
# Generate momentum signals for 20 days
start_idx = 50
num_days = 20
dates = prices_df.index[start_idx:start_idx + num_days]

# Calculate returns
returns_df = prices_df.pct_change()

# Simple momentum strategy
def calculate_momentum_signals(date, prices, returns, lookback=10):
    """Generate signals based on momentum."""
    signals = {}
    
    # Get available tickers
    tickers = ['AAPL', 'MSFT', 'GOOGL', 'TSLA', 'AMZN']
    
    for ticker in tickers:
        if ticker not in returns.columns:
            continue
        
        try:
            # Calculate momentum
            mom = returns[ticker].loc[:date].tail(lookback).mean()
            
            # Generate signal (weight proportional to momentum)
            if mom > 0.005:  # Strong positive momentum
                signals[ticker] = 0.2
            elif mom > 0.001:  # Weak positive
                signals[ticker] = 0.1
            elif mom < -0.005:  # Strong negative
                signals[ticker] = -0.1
            # else: no position
        except:
            continue
    
    return signals

# Generate signals for each day
target_weights_by_date = {}
for date in dates:
    signals = calculate_momentum_signals(date, prices_df, returns_df)
    if signals:
        target_weights_by_date[date] = signals

print(f"Generated signals for {len(target_weights_by_date)} days")
print(f"\nFirst few signals:")
for i, (date, signals) in enumerate(list(target_weights_by_date.items())[:3]):
    print(f"{date.strftime('%Y-%m-%d')}: {signals}")

# Convert to target positions
target_positions_by_date = {}
for date, weights in target_weights_by_date.items():
    positions = {}
    for ticker, weight in weights.items():
        price = prices_df.loc[date, ticker]
        shares = (portfolio_value * weight) / price
        positions[ticker] = shares
    target_positions_by_date[date] = positions

# Generate all trades
generator = ExternalTradeGenerator()
all_trades = generator.generate_multi_day_trades(
    dates=dates,
    target_positions_by_date=target_positions_by_date,
    prices_df=prices_df,
    initial_positions={}
)

print(f"\nGenerated trades for {len(all_trades)} days")
print(f"\nSample of generated trades:")
for i, (date, daily_trades) in enumerate(list(all_trades.items())[:5]):
    print(f"\n{date.strftime('%Y-%m-%d')}:")
    for ticker, trade_list in daily_trades.items():
        total_qty = sum(t['qty'] for t in trade_list)
        print(f"  {ticker}: {total_qty:6.0f} shares")

## Example 7: Run Backtest with Generated Trades

Put it all together - generate trades and run backtest.

In [ ]:
# Initialize backtester
config = BacktestConfig(
    initial_cash=1000000,
    transaction_cost_bps=10,
    max_portfolio_variance=0.01
)

backtester = Backtester(data_manager, config)

# Use the trades we generated above
print(f"Running backtest with {len(all_trades)} days of trades...")

results = backtester.run(
    start_date=dates[0],
    end_date=dates[-1],
    use_case=3,
    inputs={'external_trades': all_trades}
)

# Calculate metrics
metrics = results.calculate_metrics()
print("\n=== Performance Metrics ===")
print(f"Total Return: {metrics['total_return']:.2%}")
print(f"Sharpe Ratio: {metrics['sharpe_ratio']:.2f}")
print(f"Max Drawdown: {metrics['max_drawdown']:.2%}")
print(f"Win Rate: {metrics.get('win_rate', 0):.2%}")

# Get PnL breakdown
pnl_df = results.get_pnl_breakdown_dataframe()
print("\n=== PnL Breakdown ===")
print(pnl_df[['date', 'external_pnl', 'executed_pnl', 'overnight_pnl', 'total_pnl']].head(10))

# Get external trades summary
summary = results.get_external_trades_summary()
if not summary.empty:
    print("\n=== External Trades Summary ===")
    print(summary)

# Get execution quality (requires close prices)
exec_quality = results.get_execution_quality_analysis(prices_df)
if not exec_quality.empty:
    print("\n=== Execution Quality ===")
    print(exec_quality[['ticker', 'total_qty', 'vwap', 'avg_close', 'slippage_pct', 'execution_pnl']])

## Summary

This notebook demonstrated:

1. **Quick trade generation** using `generate_external_trades_from_signals()`
2. **Four signal types**: weights, positions, deltas, scores
3. **Multiple fills** to simulate VWAP/TWAP execution
4. **Advanced configuration** with `ExternalTradeGenerator` class
5. **Multi-day generation** from signal time series
6. **Full backtest workflow** with generated trades

### Key Takeaways

- Trade generator automatically accounts for current positions
- Multiple signal types supported for flexibility
- ADV constraints and slippage modeling for realism
- Multiple fills simulate realistic execution
- Seamlessly integrates with Use Case 3 backtesting

### Next Steps

- Customize signal generation logic for your strategy
- Experiment with different configurations
- Analyze execution quality using built-in methods
- Generate comprehensive reports with `results.generate_full_report()`